In [1]:
import sys, os
try:
    from google.colab import drive, userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

REPO_NAME = 'MistakeDetection'

if IS_COLAB:
    print("☁️ Colab rilevato.")
    if not os.path.exists('/content/drive'): drive.mount('/content/drive')

    GITHUB_USER = 'MarcoPernoVDP'
    try:
        TOKEN = userdata.get('GITHUB_TOKEN')
        REPO_URL = f'https://{TOKEN}@github.com/{GITHUB_USER}/{REPO_NAME}.git'
    except:
        REPO_URL = f'https://github.com/{GITHUB_USER}/{REPO_NAME}.git'

    ROOT_DIR = f'/content/{REPO_NAME}'
    if not os.path.exists(ROOT_DIR):
        !git clone {REPO_URL}
    else:
        %cd {ROOT_DIR}
        !git pull
        %cd /content


else:
    print("Ambiente locale rilevato.")
    ROOT_DIR = os.getcwd()
    while not os.path.exists(os.path.join(ROOT_DIR, '.gitignore')) and ROOT_DIR != os.path.dirname(ROOT_DIR):
        ROOT_DIR = os.path.dirname(ROOT_DIR)

if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)


☁️ Colab rilevato.
Mounted at /content/drive
Cloning into 'MistakeDetection'...
remote: Enumerating objects: 625, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 625 (delta 56), reused 34 (delta 16), pack-reused 533 (from 1)
Receiving objects: 100% (625/625), 85.73 MiB | 39.04 MiB/s, done.
Resolving deltas: 100% (328/328), done.


In [2]:
# Verifica ambiente
print(f"📍 Working directory: {ROOT_DIR}")
print(f"🐍 Python environment: {'Colab' if IS_COLAB else 'Local'}")

📍 Working directory: /content/MistakeDetection
🐍 Python environment: Colab


# Task 2 - Subtask 3: Hungarian Matching

Matching tra video step embeddings e task graph text embeddings usando l'algoritmo Ungherese.

## 1. Installazione Dipendenze

In [3]:
if IS_COLAB:
    !pip install scipy matplotlib seaborn -q

import numpy as np
import json
import zipfile
from pathlib import Path
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerie caricate")

✅ Librerie caricate


## 2. Configurazione Path

**Modifica questi path secondo la tua struttura:**

In [5]:
if IS_COLAB:
    # ========== CONFIGURA QUESTI PATH PER COLAB ==========
    VIDEO_FEATURES_ZIP = "/content/drive/MyDrive/MistakeDetection/hiero_all_video_steps.npz"
    TEXT_FEATURES_ZIP = "/content/drive/MyDrive/MistakeDetection/text_embeddings"
    OUTPUT_DIR = "/content/drive/MyDrive/MistakeDetection/hungarian_results"
    # ====================================================
else:
    # ========== CONFIGURA QUESTI PATH PER LOCALE ==========
    VIDEO_FEATURES_ZIP = str(Path(ROOT_DIR) / "data" / "step_embeddings.zip")
    TEXT_FEATURES_ZIP = str(Path(ROOT_DIR) / "data" / "text_embeddings.zip")
    OUTPUT_DIR = str(Path(ROOT_DIR) / "output" / "hungarian_results")
    # ====================================================

# Crea cartella output
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

print("📁 Configurazione:")
print(f"  Video features ZIP: {VIDEO_FEATURES_ZIP}")
print(f"  Text features ZIP:  {TEXT_FEATURES_ZIP}")
print(f"  Output directory:   {OUTPUT_DIR}")

📁 Configurazione:
  Video features ZIP: /content/drive/MyDrive/MistakeDetection/hiero_all_video_steps.npz
  Text features ZIP:  /content/drive/MyDrive/MistakeDetection/text_embeddings
  Output directory:   /content/drive/MyDrive/MistakeDetection/hungarian_results


## 3. Caricamento Features

In [7]:
import numpy as np

def mostra_chiavi_valori(file_path):
    print(f"--- Ispezione Contenuto: {file_path} ---")
    try:
        # Carica il file in modalità 'lazy' per non appesantire la RAM
        with np.load(file_path, allow_pickle=True) as data:
            chiavi = data.files

            if len(chiavi) == 0:
                print("Il file è vuoto.")
                return

            for chiave in chiavi:
                valore = data[chiave]
                print(f"\n🔑 CHIAVE: {chiave}")
                print(f"   Shape: {valore.shape if hasattr(valore, 'shape') else 'N/A'}")
                print(f"   Tipo: {type(valore)}")

                # Mostra il valore
                # Se è un array NumPy molto grande, mostriamo solo l'inizio
                print(f"   📄 VALORE (anteprima):\n{valore}")
                print("-" * 40)

    except Exception as e:
        print(f"Errore: {e}")

# Esecuzione sul tuo file delle video features
mostra_chiavi_valori(VIDEO_FEATURES_ZIP)

Output streaming troncato alle ultime 5000 righe.
 [-0.20339336  0.1118217   0.01230046 ...  0.19136031  0.01359259
   0.25065574]]
----------------------------------------

🔑 CHIAVE: 21_44_360p_224.mp4_1s_1s
   Shape: (8, 768)
   Tipo: <class 'numpy.ndarray'>
   📄 VALORE (anteprima):
[[-0.06456482  0.0602003  -0.02312261 ...  0.05390452  0.01575609
   0.1202472 ]
 [-0.13514583 -0.30343443  0.24083185 ...  0.14356223  0.18730265
   0.12625039]
 [ 0.3870601   0.09043605  0.18454668 ... -0.04696614  0.13278219
   0.02609073]
 ...
 [ 0.0516277  -0.11195214  0.0804874  ...  0.03609344 -0.17483884
   0.03623017]
 [ 0.09917161 -0.10258488  0.01915178 ...  0.10686067 -0.14240834
   0.23425809]
 [-0.10669055  0.01374426 -0.01359901 ...  0.19371599 -0.19744688
   0.26062542]]
----------------------------------------

🔑 CHIAVE: 21_46_360p_224.mp4_1s_1s
   Shape: (8, 768)
   Tipo: <class 'numpy.ndarray'>
   📄 VALORE (anteprima):
[[-0.07679194  0.22697937  0.15962055 ...  0.07286143  0.09009209
  

In [8]:
import json
from pathlib import Path

# 1. Definizione del path (aggiungilo alla tua configurazione esistente)
if IS_COLAB:
    ANNOTATION_JSON_PATH = "/content/drive/MyDrive/MistakeDetection/annotation_json/video_level_annotations.json"
else:
    # Esempio per locale, assumendo che sia nella cartella data/annotation_json
    ANNOTATION_JSON_PATH = str(Path(ROOT_DIR) / "data" / "annotation_json" / "video_level_annotations.json")

def carica_annotazioni(file_path):
    print(f"📂 Caricamento annotazioni da: {file_path}")
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        print("✅ Caricamento completato con successo.")

        # Mostriamo alcune informazioni di base sul JSON
        if isinstance(data, dict):
            print(f"📊 Numero di chiavi principali: {len(data.keys())}")
            print(f"🔑 Esempio chiavi: {list(data.keys())[:5]}")
        elif isinstance(data, list):
            print(f"📊 Numero di elementi nella lista: {len(data)}")

        return data

    except FileNotFoundError:
        print(f"❌ Errore: Il file '{file_path}' non è stato trovato.")
        return None
    except json.JSONDecodeError:
        print(f"❌ Errore: Il file non è un JSON valido.")
        return None
    except Exception as e:
        print(f"❌ Errore imprevisto: {e}")
        return None

# 2. Esecuzione
annotations = carica_annotazioni(ANNOTATION_JSON_PATH)

# Esempio: stampa il primo elemento se disponibile
if annotations:
    # Se il JSON è un dizionario, stampa il primo valore
    first_key = list(annotations.keys())[0] if isinstance(annotations, dict) else 0
    print(f"\n📝 Anteprima contenuto (chiave: {first_key}):")
    print(json.dumps(annotations[first_key], indent=2)[:500]) # Mostra i primi 500 caratteri

📂 Caricamento annotazioni da: /content/drive/MyDrive/MistakeDetection/annotation_json/video_level_annotations.json
✅ Caricamento completato con successo.
📊 Numero di chiavi principali: 384
🔑 Esempio chiavi: ['1_7', '1_10', '1_14', '1_19', '1_20']

📝 Anteprima contenuto (chiave: 1_7):
{
  "recording_id": "1_7",
  "activity_id": 1,
  "activity_name": "Microwave Egg Sandwich",
  "person_id": 3,
  "environment": 8,
  "has_errors": false
}


In [9]:
import numpy as np
from pathlib import Path

def carica_tutti_npz(directory_path):
    path = Path(directory_path)
    # Dizionario che conterrà: { "nome_file": { "chiave_interna": array_dati } }
    dataset_embeddings = {}

    # Verifica esistenza cartella
    if not path.exists():
        print(f"❌ La cartella non esiste: {directory_path}")
        return None

    # Recupera la lista di tutti i file .npz
    file_list = list(path.glob("*.npz"))

    if not file_list:
        print(f"⚠️ Nessun file .npz trovato in: {directory_path}")
        return None

    print(f"📂 Trovati {len(file_list)} file .npz. Inizio caricamento...")

    for file_path in file_list:
        try:
            # Carichiamo il file .npz
            # Usiamo mmap_mode='r' se i file sono enormi per non saturare la RAM
            with np.load(file_path, allow_pickle=True) as data:
                nome_file = file_path.stem  # Nome senza estensione

                # Estraiamo tutte le chiavi e i relativi array contenuti nel singolo .npz
                contenuto_file = {chiave: data[chiave] for chiave in data.files}

                dataset_embeddings[nome_file] = contenuto_file

        except Exception as e:
            print(f"❌ Errore nel caricamento di {file_path.name}: {e}")

    print(f"✅ Caricamento completato. File caricati: {len(dataset_embeddings)}")
    return dataset_embeddings

# --- ESECUZIONE ---
# Carica le embeddings di testo
text_embeddings_data = carica_tutti_npz(TEXT_FEATURES_ZIP)

# Esempio di ispezione rapida
if text_embeddings_data:
    primo_nome = list(text_embeddings_data.keys())[0]
    print(f"\n🔍 Ispezione del primo file caricato: {primo_nome}")
    for k, v in text_embeddings_data[primo_nome].items():
        if isinstance(v, np.ndarray):
            print(f"   - Chiave: '{k}' | Shape: {v.shape} | Dtype: {v.dtype}")
        else:
            print(f"   - Chiave: '{k}' | Tipo: {type(v)}")

📂 Trovati 24 file .npz. Inizio caricamento...
✅ Caricamento completato. File caricati: 24

🔍 Ispezione del primo file caricato: herbomeletwithfriedtomatoes_text
   - Chiave: 'features' | Shape: (17, 256) | Dtype: float32
   - Chiave: 'step_ids' | Shape: (17,) | Dtype: <U2
   - Chiave: 'text' | Shape: (17,) | Dtype: <U136


In [11]:
import numpy as np

# 1. Caricamento del file unico delle Video Features
print(f"Loading video features from: {VIDEO_FEATURES_ZIP}")
try:
    # Carichiamo il file e lo trasformiamo in un dizionario in memoria
    with np.load(VIDEO_FEATURES_ZIP, allow_pickle=True) as loader:
        # Creiamo il dizionario video_features usando le chiavi del file
        video_features = {k: loader[k] for k in loader.files}
    print(f"✅ Video features caricate. Numero video: {len(video_features)}")
except Exception as e:
    print(f"❌ Errore nel caricamento delle video features: {e}")
    video_features = {}

# 2. Procediamo con l'accoppiamento (il codice di prima ora funzionerà)
dataset_completo = {}
video_ids = list(video_features.keys())

print(f"🔗 Inizio accoppiamento per {len(video_ids)} video...")

count_success = 0
for vid in video_ids:
    # Recuperiamo i dati dai tre contenitori
    v_feat = video_features.get(vid)
    t_feat = text_embeddings_data.get(vid) # Caricato prima dalla cartella
    ann = annotations.get(vid)             # Caricato prima dal JSON

    # Colleghiamo solo se abbiamo tutti e tre i pezzi
    if v_feat is not None and t_feat is not None and ann is not None:
        dataset_completo[vid] = {
            'video_features': v_feat,
            'text_embeddings': t_feat,
            'annotations': ann
        }
        count_success += 1

print(f"✅ Accoppiamento completato!")
print(f"⭐ Video con dati completi: {count_success}")

# Verifica rapida su un ID specifico (es: 1_7)
test_id = "1_7"
if test_id in dataset_completo:
    print(f"\n🔍 Check {test_id}:")
    print(f"   - Video Shape: {dataset_completo[test_id]['video_features'].shape}")
    print(f"   - Text Keys: {list(dataset_completo[test_id]['text_embeddings'].keys())}")

Loading video features from: /content/drive/MyDrive/MistakeDetection/hiero_all_video_steps.npz
✅ Video features caricate. Numero video: 384
🔗 Inizio accoppiamento per 384 video...
✅ Accoppiamento completato!
⭐ Video con dati completi: 0
